## Interactive PACE data visualizer
**Motivation:** quickly see if you have PACE data available for a determined area and time period.

In [1]:
import xarray as xr
import hvplot.xarray
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import panel as pn
import numpy as np
import pandas as pd
import dask.array as da

*be sure to pip install earthaccess*

In [2]:
import earthaccess

In [3]:
auth = earthaccess.login(persist=True)

In [13]:
import datetime as dt
daterange = pn.widgets.DateRangePicker(
    name='Choose your date range:', value=(dt.date(2024,2,1),
                                          dt.date(2024,8,31))
)

text = pn.widgets.TextInput(value='Ready')

menu_items = [('PACE L3', "PACE_OCI_L3M_CHL_NRT"), 
              ('SentinelB L3', "OLCIS3B_L3m_ERR_CHL_NRT")]
menu_button = pn.widgets.MenuButton(name='Chlorophyll Data', items=menu_items, button_type='primary')

def b(event):
    text.value = f"{event.new}"
    
menu_button.on_click(b)

pn.Column(daterange,
          menu_button, text,
          height=200)

Column(height=200)
    [0] DateRangePicker(name='Choose your date range:', value=(datetime.date(2024, ...)
    [1] MenuButton(button_type='primary', items=[('PACE L3', 'PACE_OCI_L3M...], name='Chlorophyll Data')
    [2] TextInput(value='Ready')

In [30]:
tspan = (str(daterange.value[0]), 
         str(daterange.value[1])
) #choose the dates you want to search for PACE data

sname = str(text.value) #getting your data input

In [4]:
tspan = ("2024-06-20","2024-06-27")

In [31]:
if sname == "PACE_OCI_L3M_CHL_NRT":
    results = earthaccess.search_data(
    short_name="PACE_OCI_L3M_CHL_NRT",
    temporal=tspan,
    granule_name="*.DAY.*.0p1deg.*",
)

    paths = earthaccess.open(results)
elif sname == "OLCIS3B_L3m_ERR_CHL_NRT":
    granule = "*.DAY.*.9km.*"
else:
    print("No satelite data has been chosen!")

QUEUEING TASKS | :   0%|          | 0/7 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/7 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/7 [00:00<?, ?it/s]

In [8]:
tspan = ("2024-07-01", "2024-07-31")

results = earthaccess.search_data(
    short_name="PACE_OCI_L3M_CHL_NRT",
    temporal=tspan,
    granule_name="*.DAY.*.0p1deg.*",
)

paths = earthaccess.open(results)

QUEUEING TASKS | :   0%|          | 0/31 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/31 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/31 [00:00<?, ?it/s]

In [6]:
dataset = xr.open_mfdataset(
    paths,
    combine="nested",
    concat_dim="time",
    chunks= {}
)

dataset = dataset.chunk({'time': 10, 'lat': 100, 'lon': 100})
dataset

<xarray.Dataset> Size: 804MB
Dimensions:  (time: 31, lat: 1800, lon: 3600, rgb: 3, eightbitcolor: 256)
Coordinates:
  * lat      (lat) float32 7kB 89.95 89.85 89.75 89.65 ... -89.75 -89.85 -89.95
  * lon      (lon) float32 14kB -179.9 -179.9 -179.8 ... 179.8 179.9 180.0
Dimensions without coordinates: time, rgb, eightbitcolor
Data variables:
    chlor_a  (time, lat, lon) float32 804MB dask.array<chunksize=(10, 100, 100), meta=np.ndarray>
    palette  (time, rgb, eightbitcolor) uint8 24kB dask.array<chunksize=(10, 3, 256), meta=np.ndarray>
Attributes: (12/64)
    product_name:                      PACE_OCI.20240701.L3m.DAY.CHL.V2_0.chl...
    instrument:                        OCI
    title:                             OCI Level-3 Standard Mapped Image
    project:                           Ocean Biology Processing Group (NASA/G...
    platform:                          PACE
    source:                            satellite observations from OCI-PACE
    ...                                ...
    identifier_product_doi:            10.5067/PACE/OCI/L3M/CHL/2.0
    keywords:                          Earth Science > Oceans > Ocean Chemist...
    keywords_vocabulary:               NASA Global Change Master Directory (G...
    data_bins:                         748293
    data_minimum:                      0.010442081
    data_maximum:                      99.87491

As Time is not a coordinate, we must change names on it, to better investigate the data

In [7]:
dataset = dataset.assign_coords(time=pd.date_range(tspan[0], periods=len(dataset.time), freq='D'))
dataset

<xarray.Dataset> Size: 804MB
Dimensions:  (time: 31, lat: 1800, lon: 3600, rgb: 3, eightbitcolor: 256)
Coordinates:
  * lat      (lat) float32 7kB 89.95 89.85 89.75 89.65 ... -89.75 -89.85 -89.95
  * lon      (lon) float32 14kB -179.9 -179.9 -179.8 ... 179.8 179.9 180.0
  * time     (time) datetime64[ns] 248B 2024-07-01 2024-07-02 ... 2024-07-31
Dimensions without coordinates: rgb, eightbitcolor
Data variables:
    chlor_a  (time, lat, lon) float32 804MB dask.array<chunksize=(10, 100, 100), meta=np.ndarray>
    palette  (time, rgb, eightbitcolor) uint8 24kB dask.array<chunksize=(10, 3, 256), meta=np.ndarray>
Attributes: (12/64)
    product_name:                      PACE_OCI.20240701.L3m.DAY.CHL.V2_0.chl...
    instrument:                        OCI
    title:                             OCI Level-3 Standard Mapped Image
    project:                           Ocean Biology Processing Group (NASA/G...
    platform:                          PACE
    source:                            satellite observations from OCI-PACE
    ...                                ...
    identifier_product_doi:            10.5067/PACE/OCI/L3M/CHL/2.0
    keywords:                          Earth Science > Oceans > Ocean Chemist...
    keywords_vocabulary:               NASA Global Change Master Directory (G...
    data_bins:                         748293
    data_minimum:                      0.010442081
    data_maximum:                      99.87491

## Making an interactive plot

Lets plot a Log10(Chlorophyll-a) global map with PACE data

In [15]:
# Definir a projeção
projection = ccrs.PlateCarree()

# Função para criar o plot com base no índice de tempo
def create_plot(time_index):
               
    # Aplicar log10 na variável de clorofila, com Dask
    chlor_a_data = da.log10(dataset['chlor_a'].isel(time=time_index))
    
    # Criar o plot
    clorofila_plot = chlor_a_data.hvplot.quadmesh(
        'lon', 'lat', 
        projection=projection,  # Projeção do Cartopy
        cmap='jet',  # Usar a coloração do cmocean
        rasterize=True,         # Rasterização para melhor desempenho em grandes conjuntos de dados
        coastline=True,         # Adicionar linhas de costa
        geo=True,               # Definir como gráfico geoespacial
        colorbar=True,          # Mostrar colorbar
        title=f'Log10(Chlorophyll-a) [mg/m³] - {str(dataset.time[time_index].values)[0:10]}'
    )
    
    # Adicionar contorno dos continentes
    clorofila_plot = clorofila_plot.opts(
        width=800,
        height=500
    )
    
    return clorofila_plot

# Definir o intervalo de tempo
time_indexes = np.arange(len(dataset['time']))

# Criar um timepicker
slider = pn.widgets.
#slider = pn.widgets.IntSlider(name='Time index', start=0, end=len(time_indexes)-1, step=1, value=0)

# Criar o painel com o slider e o plot
interactive_plot = pn.bind(create_plot, time_index=slider)

# Layout com o slider na parte superior
layout = pn.Column(slider, interactive_plot)

# Exibir o painel
layout.servable()

Column
    [0] IntSlider(end=30, name='Time index')
    [1] ParamFunction(function, _pane=HoloViews, defer_load=False)